In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
import json
import pickle

words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('intents.json').read()
intents = json.loads(intents_file)
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")







[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

<ipython-input-1-62789c20248a>:64: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Epoch 1/200
10/10 [==============================] - 0s 1ms/step - loss: 2.2609 - accuracy: 0.0851
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1721 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1351 - accuracy: 0.1064
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0233 - accuracy: 0.2766
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9322 - accuracy: 0.3191
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7885 - accuracy: 0.4681
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7027 - accuracy: 0.4681
Epoch 8/200
10/10 [==============================] - 0s 1ms/step - loss: 1.6067 - accuracy: 0.4468
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 1.4554 - accuracy: 0.5106
Epoch 10/200
10/10 [==============================] - 0s 998us/step - loss: 1.3801 - accuracy: 0.5106
Epoch 1

Epoch 83/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0494 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0534 - accuracy: 0.9787
Epoch 85/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0299 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0598 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0500 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0169 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0263 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0858 - accuracy: 0.9787


10/10 [==============================] - 0s 1ms/step - loss: 0.0128 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0376 - accuracy: 0.9787
Epoch 166/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0194 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0188 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0147 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0264 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0637 - accuracy: 0.9574
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0335 - accuracy: 0.9787
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0076 - accuracy: 1.0000
Epoc

In [5]:
import pandas as pd
words_data = pd.read_pickle("words.pkl")
print(words_data)

["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [7]:
classes_data = pd.read_pickle("classes.pkl")
print(classes_data)

['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [2]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

In [4]:
print(model)

In [7]:
# import required libraries
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt

# Read H5 file
f = h5.File("chatbot_model.h5", "r")
# Get and print list of datasets within the H5 file
datasetNames = [n for n in f.keys()]
for n in datasetNames:
    print(n)

model_weights
optimizer_weights
